# Phase 1: Baseline Inventory Decision System

Objective:
Build a transparent, rule-based weekly inventory allocation system
for Walmart Store × Department units.

This phase establishes:
- A deterministic decision policy
- Logged decision outputs
- Evaluation metrics

No machine learning is used in this phase.


### Imports & Global Config

In [23]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Reproducibility
np.random.seed(42)

# Display
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", "{:.3f}".format)


### Load Raw Data

In [24]:
DATA_PATH = "../data/"

train = pd.read_csv(DATA_PATH + "train.csv")
features = pd.read_csv(DATA_PATH + "features.csv")
stores = pd.read_csv(DATA_PATH + "stores.csv")

print("Train:", train.shape)
print("Features:", features.shape)
print("Stores:", stores.shape)


Train: (421570, 5)
Features: (8190, 12)
Stores: (45, 3)


### Parse Dates

In [25]:
for df in [train, features]:
    df["Date"] = pd.to_datetime(train["Date"], dayfirst=True)



### Aggregate Fetaures

In [26]:
features_agg = (
    features
    .groupby(["Store", "Date", "IsHoliday"], as_index=False)
    .agg({
        "Temperature": "mean",
        "Fuel_Price": "mean",
        "CPI": "mean",
        "Unemployment": "mean",
        "MarkDown1": "sum",
        "MarkDown2": "sum",
        "MarkDown3": "sum",
        "MarkDown4": "sum",
        "MarkDown5": "sum",
    })
)


### Merge Datasets (Single Decision Table)

In [27]:
df = train.merge(
    features_agg,
    on=["Store", "Date", "IsHoliday"],
    how="left"
)

df = df.merge(
    stores,
    on="Store",
    how="left"
)

print("Merged dataset shape:", df.shape)
df.head()


Merged dataset shape: (421570, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Type,Size
0,1,1,2010-02-05,24924.500,False,48.815,2.979,217.280,7.340,6766.440,5147.700,50.820,3639.900,2737.420,A,151315
1,1,1,2010-02-12,46039.490,True,38.510,2.548,211.242,8.106,0.000,0.000,0.000,0.000,0.000,A,151315
2,1,1,2010-02-19,41595.550,False,46.425,2.883,217.401,7.340,9696.280,292.100,103.780,1133.150,6612.690,A,151315
3,1,1,2010-02-26,19403.540,False,46.630,2.561,211.320,8.106,0.000,0.000,0.000,0.000,0.000,A,151315
4,1,1,2010-03-05,21827.900,False,49.420,2.916,217.481,7.340,2460.030,0.000,3838.350,150.570,6966.340,A,151315


In [28]:
dupes = df.duplicated(subset=["Store", "Dept", "Date"]).sum()
print("Duplicate Store–Dept–Date rows:", dupes)


Duplicate Store–Dept–Date rows: 0


### Sort for Temporal Logic

In [29]:
df = df.sort_values(
    by=["Store", "Dept", "Date"]
).reset_index(drop=True)


### Sanity Checks

In [30]:
# Missing values summary
df.isna().mean().sort_values(ascending=False).head(10)


CPI            0.118
Unemployment   0.118
Fuel_Price     0.112
Temperature    0.112
MarkDown3      0.112
MarkDown2      0.112
MarkDown1      0.112
MarkDown4      0.112
MarkDown5      0.112
IsHoliday      0.000
dtype: float64

### Global Parameters

In [31]:
ROLLING_WINDOW = 12   # weeks
MIN_HISTORY = 6       # minimum weeks required


### Rolling Demand Statistics

In [32]:
df["rolling_mean"] = (
    df.groupby(["Store", "Dept"])["Weekly_Sales"]
      .transform(lambda x: x.rolling(ROLLING_WINDOW, min_periods=MIN_HISTORY).mean())
)

df["rolling_std"] = (
    df.groupby(["Store", "Dept"])["Weekly_Sales"]
      .transform(lambda x: x.rolling(ROLLING_WINDOW, min_periods=MIN_HISTORY).std())
)


### Fallback Global Department Mean

In [33]:
global_dept_mean = (
    df.groupby("Dept")["Weekly_Sales"]
      .mean()
)

global_dept_mean.head()


Dept
1   19213.485
2   43607.020
3   11793.699
4   25974.630
5   21365.584
Name: Weekly_Sales, dtype: float64

### Holiday Demand Amplification (Store-Level)

In [34]:
holiday_stats = (
    df.groupby(["Store", "IsHoliday"])["Weekly_Sales"]
      .mean()
      .unstack()
)

holiday_stats["holiday_amp"] = (
    holiday_stats[True] / holiday_stats[False]
)

holiday_stats["holiday_amp"].describe()


count   45.000
mean     1.067
std      0.046
min      0.979
25%      1.048
50%      1.066
75%      1.090
max      1.180
Name: holiday_amp, dtype: float64

### Attach Holiday Amplification

In [35]:
df = df.merge(
    holiday_stats["holiday_amp"],
    on="Store",
    how="left"
)

# Safety clamp (avoid extreme ratios)
df["holiday_amp"] = df["holiday_amp"].clip(0.8, 1.5)


### Expected Demand Estimation

In [36]:
def estimate_expected_demand(row):
    if not np.isnan(row["rolling_mean"]):
        base = row["rolling_mean"]
    else:
        base = global_dept_mean.loc[row["Dept"]]
    
    if row["IsHoliday"]:
        return base * row["holiday_amp"]
    else:
        return base

df["expected_demand"] = df.apply(
    estimate_expected_demand,
    axis=1
)


### Capacity Assumption (Simple but Explicit)
We assume each Store–Dept can stock up to 120% of its rolling mean.

In [37]:
df["capacity"] = (
    1.2 * df["rolling_mean"]
)

# Fallback if rolling_mean is missing
df["capacity"] = df["capacity"].fillna(
    1.2 * df["expected_demand"]
)


### Inventory Allocation Decision

In [38]:
df["allocated_inventory"] = np.minimum(
    df["expected_demand"],
    df["capacity"]
)


### Outcome Metrics

In [39]:
df["unmet_demand"] = np.maximum(
    df["Weekly_Sales"] - df["allocated_inventory"],
    0
)

df["overstock"] = np.maximum(
    df["allocated_inventory"] - df["Weekly_Sales"],
    0
)

# Service level: defined only when demand > 0
df["service_level"] = np.where(
    df["Weekly_Sales"] > 0,
    (df["Weekly_Sales"] - df["unmet_demand"]) / df["Weekly_Sales"],
    np.nan
)


### Clean Decision Log

In [40]:
decision_log = df[
    [
        "Store", "Dept", "Date",
        "Weekly_Sales",
        "expected_demand",
        "allocated_inventory",
        "unmet_demand",
        "overstock",
        "service_level",
        "IsHoliday"
    ]
].copy()

decision_log.head()


,Store,Dept,Date,Weekly_Sales,expected_demand,allocated_inventory,unmet_demand,overstock,service_level,IsHoliday
0,1,1,2010-02-05,24924.500,19213.485,19213.485,5711.015,0.000,0.771,False
1,1,1,2010-02-12,46039.490,20484.701,20484.701,25554.789,0.000,0.445,True
2,1,1,2010-02-19,41595.550,19213.485,19213.485,22382.065,0.000,0.462,False
3,1,1,2010-02-26,19403.540,19213.485,19213.485,190.055,0.000,0.990,False
4,1,1,2010-03-05,21827.900,19213.485,19213.485,2614.415,0.000,0.880,False


### Aggregate Evaluation Metrics

In [41]:
metrics = {
    "avg_service_level": decision_log["service_level"].mean(skipna=True),
    "total_unmet_demand": decision_log["unmet_demand"].sum(),
    "total_overstock": decision_log["overstock"].sum(),
    "mean_weekly_allocation": decision_log["allocated_inventory"].mean(),
    "allocation_volatility": decision_log["allocated_inventory"].std(),
    "fraction_zero_demand": (decision_log["Weekly_Sales"] == 0).mean()
}

metrics_df = pd.DataFrame.from_dict(metrics, orient="index", columns=["value"])
metrics_df



,value
avg_service_level,0.923
total_unmet_demand,516324186.974
total_overstock,563918567.004
mean_weekly_allocation,16094.156
allocation_volatility,21895.761
fraction_zero_demand,0.000


### Save Outputs

In [42]:
OUTPUT_PATH = "../outputs/"

decision_log.to_csv(
    OUTPUT_PATH + "phase1_decisions.csv",
    index=False
)

metrics_df.to_csv(
    OUTPUT_PATH + "phase1_metrics.csv"
)


In [43]:
decision_log["service_level"].describe()


count   420212.000
mean         0.923
std          0.285
min        -71.600
25%          0.912
50%          1.000
75%          1.000
max          1.000
Name: service_level, dtype: float64

## Phase 1 Complete

Phase 1 established a deterministic, interpretable baseline inventory decision system operating under realistic demand heterogeneity and capacity constraints. The system produces stable but inefficient allocations, exhibiting significant decision volatility despite acceptable average service levels. This behavior provides a suitable foundation for controlled degradation analysis.

This system will be intentionally stressed in Phase 2
to expose decision degradation.